In [6]:
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
df = pd.read_csv('detections-20141206.csv', delimiter="|", nrows=1000)

In [3]:
df.head()

,created_at,audio_source_id,track_id
0,2014-10-01 00:00:35 UTC,9088,690
1,2014-10-01 00:00:37 UTC,288,41082
2,2014-10-01 00:00:38 UTC,345,148509
3,2014-10-01 00:00:38 UTC,493,33022
4,2014-10-01 00:00:36 UTC,1724,4539


## Os índices das rádios são esparsos?

Verificando se posso usar o próprio ID como identificação da dimensão ou se precisarei mexer nisso:

In [17]:
sources = set(df['audio_source_id'])
n_sources = len(sources)
print("# de ID (dimensões): {}".format(n_sources))
print("ID mínimo: {}".format(min(sources)))
print("ID máximo: {}".format(max(sources)))

# de ID (dimensões): 801
ID mínimo: 3
ID máximo: 9352


É... os ID são esparsos. Isso significa que é melhor re-rotular as dimensões:

In [15]:
a = np.array(list(sources))
b = np.sort(a)

[   3    6    9   19   27   29   33   36   38   41   42   44   45   51   59
   62   63   69   78   88   93  109  111  114  128  132  146  152  157  168
  194  196  203  210  215  216  217  221  225  227  232  235  242  249  250
  251  269  277  288  301  306  313  319  321  323  327  336  345  349  351
  353  367  371  373  375  379  383  387  389  399  408  417  419  421  436
  445  450  459  460  469  477  482  489  493  499  501  502  507  526  527
  532  549  553  562  584  589  598  611  614  620  626  636  641  643  645
  653  655  686  690  691  693  695  697  719  724  725  732  750  753  754
  755  783  785  786  790  791  803  808  813  817  818  858  868  873  874
  884  900  901  914  919  922  925  937  938  939  962  968  969  975  987
  994 1005 1021 1024 1027 1029 1036 1069 1091 1099 1102 1105 1108 1110 1111
 1122 1124 1140 1159 1162 1165 1178 1193 1209 1216 1217 1229 1238 1241 1242
 1247 1250 1256 1265 1266 1276 1277 1288 1293 1296 1307 1308 1311 1325 1329
 1345 1347 1

In [43]:
n_sources = len(sources)
reindexer = dict(zip(b,range(n_sources)))

In [44]:
reindexer

{3: 0,
 6: 1,
 9: 2,
 19: 3,
 27: 4,
 29: 5,
 33: 6,
 36: 7,
 38: 8,
 41: 9,
 42: 10,
 44: 11,
 45: 12,
 51: 13,
 59: 14,
 62: 15,
 63: 16,
 69: 17,
 78: 18,
 88: 19,
 93: 20,
 109: 21,
 111: 22,
 114: 23,
 128: 24,
 132: 25,
 146: 26,
 152: 27,
 157: 28,
 168: 29,
 194: 30,
 196: 31,
 203: 32,
 210: 33,
 215: 34,
 216: 35,
 217: 36,
 221: 37,
 225: 38,
 227: 39,
 232: 40,
 235: 41,
 242: 42,
 249: 43,
 250: 44,
 251: 45,
 269: 46,
 277: 47,
 288: 48,
 301: 49,
 306: 50,
 313: 51,
 319: 52,
 321: 53,
 323: 54,
 327: 55,
 336: 56,
 345: 57,
 349: 58,
 351: 59,
 353: 60,
 367: 61,
 371: 62,
 373: 63,
 375: 64,
 379: 65,
 383: 66,
 387: 67,
 389: 68,
 399: 69,
 408: 70,
 417: 71,
 419: 72,
 421: 73,
 436: 74,
 445: 75,
 450: 76,
 459: 77,
 460: 78,
 469: 79,
 477: 80,
 482: 81,
 489: 82,
 493: 83,
 499: 84,
 501: 85,
 502: 86,
 507: 87,
 526: 88,
 527: 89,
 532: 90,
 549: 91,
 553: 92,
 562: 93,
 584: 94,
 589: 95,
 598: 96,
 611: 97,
 614: 98,
 620: 99,
 626: 100,
 636: 101,
 641: 102,
 

## Todas as rádios são significativas?

In [16]:
# TODO: verificar se há um subconjunto de rádios mais frequentes.
# Se sim, posso usar, por exemplo, o critério de Pareto (arbitrário) para reduzir a quantidade de dimensões

In [38]:
aggregated = df.groupby(['audio_source_id']).agg(['count'])
aggregated.head()

,created_at,track_id
,count,count
audio_source_id,,
3,1,1
6,2,2
9,2,2
19,1,1
27,2,2


In [39]:
aggregated.describe()

,created_at,track_id
,count,count
count,801.000000,801.000000
mean,1.248439,1.248439
std,0.489337,0.489337
min,1.000000,1.000000
25%,1.000000,1.000000
50%,1.000000,1.000000
75%,1.000000,1.000000
max,4.000000,4.000000


Parece que há bem poucas ocorrências por rádio, de modo que não dá para descartar nenhuma. (ISSO PARA 1000 AMOSTRAS!!!!)

## Reindexando as rádios

In [45]:
def reindex(source_id):
    return reindexer[source_id]

df['source_id'] = df['audio_source_id'].apply(reindex)

In [46]:
df.head()

,created_at,audio_source_id,track_id,source_id
0,2014-10-01 00:00:35 UTC,9088,690,782
1,2014-10-01 00:00:37 UTC,288,41082,48
2,2014-10-01 00:00:38 UTC,345,148509,57
3,2014-10-01 00:00:38 UTC,493,33022,83
4,2014-10-01 00:00:36 UTC,1724,4539,241


Com isso os índices das rádios compõem um conjunto compacto.

## Espectro de uma trilha de áudio (track)

In [51]:
track_spectrum = np.zeros(shape=(n_sources))

In [68]:
#def construct_track_spectrum(track_id):
track_id = 690
track = df[df['track_id'] == track_id].groupby(['source_id']).agg(['count'])['audio_source_id']
track

,count
source_id,
577,1
622,1
782,1


In [70]:
absolute_freq = track.sum()
absolute_freq

count    3
dtype: int64

In [71]:
track = track/absolute_freq

In [72]:
track

,count
source_id,
577,0.333333
622,0.333333
782,0.333333


In [80]:
for idx, value in track.itertuples():
    track_spectrum[idx] = value

In [81]:
track_spectrum

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [120]:
def construct_track_spectrum(track_id):
    spectrum = np.zeros(shape=(n_sources))
    track = df[df['track_id'] == track_id].groupby(['source_id']).agg(['count'])['audio_source_id']
    absolute_freq = track.sum()
    track = track/absolute_freq
    for idx, value in track.itertuples():
        spectrum[idx] = value
    return spectrum

In [84]:
track_690 = construct_track_spectrum(690)

In [86]:
np.shape(track_690)

(801,)

## Outra forma de gerar o espectro, talvez mais rápida

In [88]:
aggregate = df[['created_at', 'track_id', 'source_id']].groupby(['track_id', 'source_id']).agg(['count'])

In [109]:
# TODO: prosseguir...

## Gerando o espectro de todos os tracks 

### Identificando os tracks existentes

A ordenação é útil porque o k-Means me devolve os rótulos dos clusters num vetor alinhado com o vetor de tracks, X, mais abaixo. Desse modo, posso associar univocamente e facilmente o índice do cluster com o track_id

In [142]:
track_ids = np.sort(np.array(list(set(df['track_id']))))

In [143]:
print("Há {} músicas".format(len(track_ids)))

Há 792 músicas


In [144]:
track_ids[:10]

array([135, 195, 204, 209, 227, 311, 537, 594, 614, 618])

### Construindo os espectros

In [132]:
tracks = []
for t in track_ids:
    tracks.append((t, construct_track_spectrum(t)))

In [134]:
tracks_dict = dict(tracks)

### Persistindo os espectos

In [137]:
import pickle

pickle.dump(tracks_dict, file('X_dict.pickle', 'wb'), pickle.HIGHEST_PROTOCOL)

O objetivo desse notebook era esse: escrever as tracks num formato fácil de trabalhar

###  Formato 2: array X

In [145]:
X = np.ndarray(shape=(1000,n_sources))

In [146]:
i = 0
for t in track_ids:
    X[i,:] = construct_track_spectrum(t)

In [147]:
pickle.dump(X, file('X.pickle', 'wb'), pickle.HIGHEST_PROTOCOL)

### Persistindo os ID das tracks em ordem

In [149]:
pickle.dump(track_ids, file('track_ids.pickle', 'wb'), pickle.HIGHEST_PROTOCOL)